In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
import io
import typing as t
import numpy as np
from math import ceil
from tqdm import tqdm
from time import time
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow.keras import layers
# Helper libraries
import numpy as np
import os

print(tf.__version__)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


2.15.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

BATCH_SIZE = 2  # per replica batch size

# initialize tf.distribute.MirroredStrategy
strategy = tf.distribute.MirroredStrategy(devices=None)
GLOBAL_BATCH_SIZE = strategy.num_replicas_in_sync * BATCH_SIZE

print(f'Number of devices: {strategy.num_replicas_in_sync}')

Number of devices: 1


In [ ]:
# Simulate multiple CPUs with virtual devices
N_VIRTUAL_DEVICES = 2
physical_devices = tf.config.list_physical_devices("CPU")
tf.config.set_logical_device_configuration(
    physical_devices[0], [tf.config.LogicalDeviceConfiguration() for _ in range(N_VIRTUAL_DEVICES)])

RuntimeError: Virtual devices cannot be modified after being initialized

In [ ]:
print("Available devices:")
for i, device in enumerate(tf.config.list_logical_devices()):
  print("%d) %s" % (i, device))

Available devices:
0) LogicalDevice(name='/device:CPU:0', device_type='CPU')


In [ ]:
import tensorflow as tf

batch_size = 42
# Define the dimensions of your images
img_height = 224
img_width = 224

In [ ]:
# Create a distributed strategy
strategy = tf.distribute.MirroredStrategy()
# Load and distribute the dataset
with strategy.scope():
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        '/content/drive/MyDrive/Image_segregated/train',
        batch_size=batch_size,
        image_size=(img_height, img_width),
        validation_split=0.2,
        subset='training',
        seed=42
    )
    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        '/content/drive/MyDrive/Image_segregated/val',
        batch_size=batch_size,
        image_size=(img_height, img_width),
        validation_split=0.2,
        subset='validation',
        seed=42
    )
    test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        '/content/drive/MyDrive/Image_segregated/test',
        batch_size=batch_size,
        image_size=(img_height, img_width)
    )



Found 23755 files belonging to 2 classes.
Using 19004 files for training.
Found 10547 files belonging to 2 classes.
Using 2109 files for validation.
Found 12270 files belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models, optimizers

# Define the number of classes (binary classification)
num_classes = 2
epochs=10

# Distributed training setup
strategy = tf.distribute.MirroredStrategy()

# Define InceptionV3 model
with strategy.scope():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x)  # Use a single neuron with sigmoid activation for binary classification
    model = models.Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(optimizer=optimizers.Adam(),
                  loss='binary_crossentropy',  # Use binary crossentropy for binary classification
                  metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
864/864 [==============================] - 9395s 11s/step - loss: 0.1846 - accuracy: 0.9333 - val_loss: 5.0496 - val_accuracy: 0.1522
Epoch 2/10
864/864 [==============================] - 9134s 11s/step - loss: 0.0928 - accuracy: 0.9692 - val_loss: 5.8861 - val_accuracy: 0.0602
Epoch 3/10
864/864 [==============================] - 9122s 11s/step - loss: 0.0756 - accuracy: 0.9754 - val_loss: 5.9444 - val_accuracy: 0.1133
Epoch 4/10
340/864 [==========>...................] - ETA: 1:28:30 - loss: 0.0641 - accuracy: 0.9785

KeyboardInterrupt: 

In [ ]:
model_path = '/content/drive/MyDrive/tensorflow_model/inceptionv3_tensorflow_model'


In [ ]:
pytorch_model_path = '/content/drive/MyDrive/pytorch_model/inceptionv3_pytorch_model.pth'


In [ ]:
tensorflow_model.save(model_path)
torch.save(pytorch_model.state_dict(), pytorch_model_path)

NameError: name 'tensorflow_model' is not defined

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the PyTorch model architecture
class CustomInceptionV3(nn.Module):
    def __init__(self):
        super(CustomInceptionV3, self).__init__()
        self.base_model = models.inception_v3(pretrained=False)
        # Modify the classifier to match the output size (1 neuron for binary classification)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 1)

    def forward(self, x):
        return self.base_model(x)

# Initialize the PyTorch model
pytorch_model = CustomInceptionV3()

# Load weights from the TensorFlow model into the PyTorch model
# You'll need to manually specify the corresponding layers to load the weights
# Example: pytorch_model.base_model.load_state_dict(tf_model_weights)

# Save the PyTorch model in .pth format
torch.save(pytorch_model.state_dict(), 'inceptionv3_model.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
